# Getting Started with Patra Toolkit

This notebook serves as a quickstart guide to help you learn how to:

- Load and preprocess an example image
- Perform image classification with a pretrained **ResNet50** model from PyTorch
- Generate a comprehensive Model Card using the **Patra Toolkit**

By the end of this tutorial, you'll have a Model Card (in JSON format) that captures key metadata about your model and its prediction.

---

## 1. Environment Setup

### 1.1 Install Required Packages
Run the following cell to install the required packages:
- `torch` and `torchvision` for the model and image processing
- `patra_toolkit` for creating the Model Card
- `Pillow` for image handling
- `scikit-learn` (if needed) for additional utilities

---


In [21]:
!pip install torch torchvision patra_toolkit Pillow scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


### 1.2 Import Dependencies

Below, we import the necessary libraries. We also add the repository root to `sys.path` so that the latest local version of `patra_toolkit` is imported (if needed).

---

In [22]:
import os
import sys
import io
import json
import logging
import tempfile
import requests

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from PIL import Image

# Import Patra Toolkit components
from patra_toolkit import ModelCard, AIModel

# Set logging level
logging.basicConfig(level=logging.INFO)

# Optionally, add repo root to sys.path if working with a local patra_toolkit repo
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

---
## 2. Load and Preprocess an Example Image

We'll download an example image from a URL. Then, we'll apply the same preprocessing as required by ResNet50:
- Resize to 256 pixels on the smaller side
- Center-crop to 224×224
- Convert to a tensor and normalize using the ImageNet statistics

---


In [23]:
# URL of an example image (you can choose any valid image URL)
image_url = "https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg"  # example: a picture of a pug

# Download the image
response = requests.get(image_url)
response.raise_for_status()

# Open the image with PIL
image = Image.open(io.BytesIO(response.content)).convert("RGB")

# Define the transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # Normalization parameters from ImageNet
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

input_tensor = preprocess(image)
# Create a mini-batch as expected by the model
input_batch = input_tensor.unsqueeze(0)

---
## 3. Image Prediction with Pretrained ResNet50

We load the pretrained ResNet50 model and perform inference on the preprocessed image. We'll then decode the top prediction using the default weights.

---


In [24]:
import os
import certifi

os.environ["SSL_CERT_FILE"] = certifi.where()

# Load pretrained ResNet50 model
# torchvision.models.ResNet50_Weights.DEFAULT is available in newer torchvision releases
weights = torchvision.models.ResNet50_Weights.DEFAULT
model = torchvision.models.resnet50(weights=weights)
model.eval()  # set to evaluation mode

# Perform inference
with torch.no_grad():
    output = model(input_batch)

# Get probabilities using softmax
probabilities = F.softmax(output[0], dim=0)

# Get the top 5 predictions
top5_prob, top5_catid = torch.topk(probabilities, 5)

# Use the weights metadata to map category IDs to labels
categories = weights.meta["categories"]

## 4. Model Card Generation with Patra Toolkit

Next, we'll create a Model Card capturing key metadata about the model.
Since we're using a pretrained ResNet50, we'll record its details and top prediction.


In [25]:
# Create a ModelCard instance
mc = ModelCard(
    name="ResNet50",
    version="0.1",
    short_description="A pretrained ResNet50 image classifier",
    full_description="This model card demonstrates using a pretrained ResNet50 model from PyTorch",
    keywords="resnet50, pytorch, image classification, patra, pretrained",
    author="neelk",
    input_type="Image",
    category="classification",
    foundational_model="None"
)

# For demonstration, we set input_data as the example image URL
mc.input_data = image_url


### 4.1 Create an AIModel Instance

We attach an AIModel instance to the ModelCard with details about the model.

---

In [26]:
ai_model = AIModel(
    name="ResNet50",
    version="0.1",
    description="Pretrained ResNet50 model from torchvision for image classification.",
    owner="Neelesh Karthikeyan",
    location="",  # will be updated after model submission
    license="BSD-3 Clause",
    framework="pytorch",
    model_type="cnn",
    test_accuracy=0.0  # As we're not training, we leave this as 0.0
)

# Attach the AIModel to the ModelCard
mc.ai_model = ai_model


### 4.2 Automatically Capture Environment Requirements

This function parses the environment to capture installed packages.

---

In [27]:
mc.populate_requirements()
print("Requirements captured:")
print(mc.model_requirements[:5])  # print the first five requirements

Requirements captured:
['absl-py==2.1.0', 'aiohappyeyeballs==2.5.0', 'aiohttp==3.11.13', 'aiosignal==1.3.2', 'annotated-types==0.7.0']


---
## 5. Validate and Save the Model Card

We now validate the Model Card against the default schema. If valid, we save it locally as a JSON file.

---

In [28]:
if mc.validate():
    mc.save("imagenet_mc.json")
    print("Model Card validated successfully and saved as 'imagenet_mc.json'.")
else:
    print("Model Card validation failed.")


INFO:root:Model card validated successfully.
INFO:root:Model card saved to imagenet_mc.json.


Model Card validated successfully and saved as 'imagenet_mc.json'.


---
## 6. Submit the Model and Artifact to the Patra Server and Model Store

For demonstration purposes, we assume the Patra server is running locally at `http://127.0.0.1:5002`
and that the model store (e.g., Hugging Face) is configured accordingly.

**Note:**
- When submitting a model, we need to serialize it.
- For PyTorch, common serialization formats include `"pt"` (using `state_dict()`) or `"onnx"`.
- In this example, we use `"pt"`.
- Similarly, we can submit an artifact (for example, the input image file).

---

In [29]:
# Submit the model to the Patra server
mc.submit_model(
    patra_server_url="http://127.0.0.1:5002",
    model=model,
    file_format="pt",
    model_store="github"
)

INFO:root:PID generated: neelk-resnet50-0.1


Repository 'neelk-resnet50-0.1' created successfully.
Initialized empty Git repository in /private/var/folders/d7/zwq9fkgs65xdfbrv7v00g8dc0000gn/T/neelk-resnet50-0.1rm1ihed5/.git/


remote: warning: See https://gh.io/lfs for more information.        
remote: warning: File neelk-resnet50-0.1.pt is 97.79 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
To https://github.com/nee1k/neelk-resnet50-0.1.git
 * [new branch]      main -> main
INFO:root:Model stored at: https://github.com/nee1k/neelk-resnet50-0.1/blob/main/neelk-resnet50-0.1.pt
INFO:root:Model card validated successfully.
INFO:root:Model card submitted successfully.


{'message': 'Successfully uploaded the model card',
 'model_card_id': 'neelk-resnet50-0.1'}

In [30]:
mc.submit_artifact(
    patra_server_url="http://127.0.0.1:5002",
    artifact_path="labels.txt",
    model_store="github"
)

INFO:root:PID generated: neelk-resnet50-0.1


Repository 'neelk-resnet50-0.1' already exists. Using existing repository.
Initialized empty Git repository in /private/var/folders/d7/zwq9fkgs65xdfbrv7v00g8dc0000gn/T/neelk-resnet50-0.13db_y19k/.git/


From https://github.com/nee1k/neelk-resnet50-0.1
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main
To https://github.com/nee1k/neelk-resnet50-0.1.git
   3f8a409..52cf2df  main -> main
INFO:root:Artifact stored at: https://github.com/nee1k/neelk-resnet50-0.1/blob/main/labels.txt
INFO:root:Model card validated successfully.
INFO:root:Model card submitted successfully.


{'message': 'Model card already exists', 'model_card_id': 'neelk-resnet50-0.1'}

---
## 7. Conclusion

In this notebook, we have:
- Loaded and preprocessed an example image.
- Performed image prediction using a pretrained ResNet50 model from PyTorch.
- Created and populated a Model Card using the Patra Toolkit.
- Validated and saved the Model Card.
- Submitted the model and an artifact to the Patra server (assuming the server is running).

This serves as a foundation for creating more transparent and accountable AI solutions with detailed metadata tracking using the Patra Toolkit.